# Notebook from October 2, 2020

Focus on getting quantitative evaluation.

In [1]:
from replay_trajectory_classification import ClusterlessDecoder
from replay_trajectory_classification.state_transition import estimate_movement_var
from replay_trajectory_classification.misc import NumbaKDE

In [2]:
def get_decoder():
    model_kwargs = {
    'bandwidth': np.array([1.0, 1.0, 1.0, 1.0, 12.5]) # amplitude 1, amplitude 2, amplitude 3, amplitude 4, position
    }

    decoder = ClusterlessDecoder(model_kwargs=model_kwargs)
    
    return decoder

In [8]:
from neural_dimensionality_reduction import get_formatted_data
import neural_dimensionality_reduction as ndr
import numpy as np

In [15]:
multiunits, position = get_formatted_data(ndr.data_dir, tetrode_id=6)
print(position.shape)
print(multiunits.shape)

(615851,)
(615851, 4, 1)


/nix/store/hb4scs94q1fl6hypm6bz2bbi91dxqci7-python3-3.8.5-env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3337: DtypeWarning: Columns (11,12,13,15) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [43]:
def get_transform(tetrode_id, sigma_function=lambda x: 1, spike_preprocess = None, weights=None):
    multiunits, position = get_formatted_data(ndr.data_dir, tetrode_id=tetrode_id)
    
    multiunits_reshape = multiunits.reshape((multiunits.shape[0], 4))
    spikes_bool = np.logical_not(np.isnan(multiunits_reshape)).any(axis=1) 
    spikes_idx = np.argwhere(spikes_bool).reshape((-1))
    print(spikes_idx.shape)
    
    raw_spikes = multiunits_reshape[spikes_idx]
    print(raw_spikes.shape)
    
    if spike_preprocess is None:
        spikes = raw_spikes
    else:
        spikes = spike_preprocess(raw_spikes)

    som = ndr.reduction.SelfOrganizingMap3D((16, 16, 16, 4), mu=0, sigma=1, sigma_function=sigma_function, weights=weights)

    for i in range(spikes.shape[0]):
        som.train1(spikes[i])

    transformed, _ = ndr.reduction.get_transformed_colors_3d(som, spikes)

    multiunits_transformed = np.empty((multiunits.shape[0],3))
    multiunits_transformed[:] = np.nan
    multiunits_transformed[spikes_idx] = transformed
    
    multiunits_transformed_formatted = multiunits_transformed.reshape((multiunits_transformed.shape[0], multiunits_transformed.shape[1], 1))
    
    return multiunits_transformed_formatted, position

In [44]:
shape = (16, 16, 16, 4)
weights = np.random.normal(1000, 500, shape)
transformed, position = get_transform(6, weights=weights)

(44064,)
(44064, 4)


In [42]:
transformed.shape

(615851, 3)